In [22]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import *
from tensorflow.keras.preprocessing.text import Tokenizer as tk
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

In [28]:
df=pd.read_csv("IMDB Dataset.csv")

In [29]:
df.shape

(50000, 2)

In [30]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [14]:
#Since this dataset is available to use from keras datasets

from tensorflow.keras.datasets import imdb

# the data, shuffled and split between train and test sets
(xtr, ytr), (xte, yte) = imdb.load_data()

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
C:\Users\37086\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
C:\Users\37086\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) i

In [17]:
print("Training Data shape: ", xtr.shape)
print("Training Label shape: ", ytr.shape)
print("Testing Data shape: ", xte.shape)
print("Testing Label shape: ", yte.shape)

Training Data shape:  (25000,)
Training Label shape:  (25000,)
Testing Data shape:  (25000,)
Testing Label shape:  (25000,)


In [31]:
encoder = LabelEncoder()

df['sentiment'] = encoder.fit_transform(df['sentiment'])

In [32]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [33]:
X = np.array(df.drop(columns=['sentiment'])).reshape(len(df),)
y = np.array(df['sentiment'])

In [34]:
xtr , xte , ytr, yte = train_test_split(X,y,shuffle=True,test_size = 0.1,stratify=y)

In [35]:
tok=tk(num_words=10000, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True, split=' ',char_level=False, oov_token='Farid')

In [36]:
tok.fit_on_texts(xtr)

In [37]:
xtr1 = tok.texts_to_sequences(xtr)
xte1 = tok.texts_to_sequences(xte)

In [38]:
xtr2 = pad_sequences(xtr1,1000, padding='post',truncating='post')
xte2 = pad_sequences(xte1,1000, padding='post', truncating='post')

In [39]:
xtr3 = np.flip(xtr2, 1)
xte3 = np.flip(xte2, 1)

In [40]:
print(xtr3.shape)
print(xte3.shape)

(45000, 1000)
(5000, 1000)


In [41]:
embedding_vecor_length = 50
max_review_length = 1000
NUM_WORDS = 10000

model = Sequential()
model.add(Embedding(NUM_WORDS, embedding_vecor_length, input_length=max_review_length))
model.add(Conv1D(filters=100, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D())
model.add(GRU(50, return_sequences=True))
model.add(GRU(50))
model.add(Dropout(0.5))
model.add(Dense(100,activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [42]:
model.compile(optimizer='adam', loss='binary_crossentropy',metrics='accuracy')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1000, 50)          500000    
_________________________________________________________________
conv1d (Conv1D)              (None, 1000, 100)         15100     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 500, 100)          0         
_________________________________________________________________
gru (GRU)                    (None, 500, 50)           22800     
_________________________________________________________________
gru_1 (GRU)                  (None, 50)                15300     
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense (Dense)                (None, 100)               5

In [43]:
model.fit(xtr3,ytr,epochs=5,batch_size=100,validation_data = (xte3,yte),shuffle=True)

Epoch 1/5
450/450 [==============================] - 713s 2s/step - loss: 0.5414 - accuracy: 0.6876 - val_loss: 0.2753 - val_accuracy: 0.8934
Epoch 2/5
450/450 [==============================] - 686s 2s/step - loss: 0.2212 - accuracy: 0.9185 - val_loss: 0.2618 - val_accuracy: 0.9052
Epoch 3/5
450/450 [==============================] - 694s 2s/step - loss: 0.1585 - accuracy: 0.9452 - val_loss: 0.2586 - val_accuracy: 0.9052
Epoch 4/5
450/450 [==============================] - 700s 2s/step - loss: 0.1137 - accuracy: 0.9624 - val_loss: 0.2619 - val_accuracy: 0.9024
Epoch 5/5
450/450 [==============================] - 1465s 3s/step - loss: 0.0836 - accuracy: 0.9734 - val_loss: 0.2834 - val_accuracy: 0.9024


In [44]:
print("Accuracy: ", np.round(model.evaluate(xte3,yte,verbose=0)[1]*100,3),"%")

Accuracy:  90.24 %


In [46]:
review_num = 4
random_indexs = []
random_reviews = []

for _ in range(review_num):
    rand_index = np.random.randint(0,xte.shape[0])
    random_indexs.append(rand_index)
    random_reviews.append(xte3[rand_index])

random_reviews = np.array(random_reviews)
predictions = (model.predict(random_reviews) > 0.5).astype('int32').reshape(review_num,)

for i,index in enumerate(random_indexs):
    print("Review:  " , xte[index],end="\n\n\n")
    actual_review = "Positive" if yte[index] == 1 else "Negative"
    predicted_review = "Positive" if predictions[i] == 1 else "Negative"
    print("Actual review status: ",actual_review)
    print("Predicted review status: ",predicted_review,end='\n\n')

Review:   Week after week these women just sweep all the men of their feet. Get real. None of these women are "Knockouts". Carrie (Sarah Jessica Parker) looks like the type of woman men would pick up at !:45am before the bar closed after their vision and standards were equally impaired by ten or eleven martinis. Yet she's the queen bee, a super-sexy man-killer. The other three don't fare much better. And their constant foul mouthed comments.....not to mention that they jump in and out of bed with strange men and never catch a disease. This show is pathetic .and creepy.I don't think any man would be terribly attracted to any of these women, even if he popped Viagra like Tic Tacs while on shore leave.


Actual review status:  Negative
Predicted review status:  Negative

Review:   The reasons to watch this knock off... err... tribute to a great movie called Se7en: - It's on while your channel surfing and there's nothing else on. - Someone pays you to watch it.<br /><br />Do yourself a fav